<a href="https://colab.research.google.com/github/Wiickz/MAT421/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Section 1.1: Introduction to Linear Algebra

The unique approach to solving mathematical problems often found in linear algebra extends well to the fields of numerical analysis and data science. Handling large quantities of data using standard algebraic techniques would be time-consuming, even with the help of computers. However, by applying concepts of linear algebra, even larger systems can be handled quite well:

In [ ]:
### Excerpt from own work in MAT 423: Numerical Analysis I
### LDL^T factorization to solve a system of linear equations represented as a matrix equation Ax = b

import numpy as np

def p4(A, b):
    '''Solve a linear system Ax = b using LDL^T factorization'''
    n = A.shape[0]
    L = np.zeros((n, n))
    D = np.zeros(n)
    for i in range(n):
        s = 0
        for k in range(i):
            s += D[k] * L[i, k] ** 2
        D[i] = A[i, i] - s
        L[i, i] = 1
        for j in range(i+1, n):
            s = 0
            for k in range(i):
                s += D[k] * L[i, k] * L[j, k]
            L[j, i] = (A[j, i] - s) / D[i]
    print('L =', L)
    print('D =', D)
    y = np.zeros(n)
    y[0] = b[0]
    for i in range(1, n):
        s = 0
        for k in range(i):
            s += L[i, k] * y[k]
        y[i] = b[i] - s
    z = np.zeros(n)
    for i in range(n):
        z[i] = y[i] / D[i]
    x = np.zeros(n)
    x[n-1] = z[n-1]
    for i in range(n-2, -1, -1):
        s = 0
        for k in range(i+1, n):
            s += L[k, i] * x[k]
        x[i] = z[i] - s

    return x

# sample systems
A1 = np.array([[4, 1, 1, 1], [1, 3, -1, 1], [1, -1, 2, 0], [1, 1, 0, 2]])
b1 = np.array([0.65, 0.05, 0, 0.5])
x1 = p4(A1, b1)
print('x1 =', x1)

A2 = np.array([[6, 2, 1, -1], [2, 4, 1, 0], [1, 1, 4, -1], [-1, 0, -1, 3]])
b2 = np.array([0, 7, -1, -2])
x2 = p4(A2, b2)
print('x2 =', x2)

# a bit rough on the edges in terms of output presentation, but it does work

## Section 1.2: Elements of Linear Algebra

We're getting a little ahead of ourselves here. Let's start with some of the basic concepts:

### Linear Spaces (and the basis)

A linear space can be defined as a set of vectors whose elements can be multiplied by scalars and/or added together to create a result vector that also exists within the same set. Linear spaces have linear subspaces that exist under similar conditions: as a subset of a linear space, all multiplication/addition operations on the comprising vectors result in vectors that exist within the subset (and thus the set).

As an example, suppose we have a linear space consisting of vectors [x, y, z]. Multiplying one of these vectors, or adding two (or three) of them together, results in a value that still depends entirely on the basis vectors (e.g. 2y or x+z). A subspace can be created consisting of only one or two of the basis vectors (e.g. [x, z]) that still follows the same rules (with multiplication or addition operations still creating results such as 4z+x).

### Linear Independence

Suppose we have such a linear space with a set of vectors like the one above. Sometimes, one vector can be expressed in terms of one or more of the other vectors:

In [ ]:
import numpy as np

x = np.array([1, 2, 3, 4])
y = np.array([2, 4, 6, 8])

print("x: ", x)
print("y: ", y)
print("2x: ", 2*x)

If this is the case, then the vectors are linearly dependent - but if not, then we can say the vectors are linearly independent. A scalar value of zero is an exception here, as any vector, no matter its magnitude, multiplied by zero will create a zero vector.

### Orthogonality

Orthogonality, grossly simplified, is a measure of vector perpendicularity. If the inner product of two vectors is zero, then they are considered to be orthogonal. However, if you have a list of vectors, and (a) for each combination of two vectors, their inner product is zero, and (b) the norm of each vector is equal to 1, then the list is said to be orthonormal:

In [ ]:
import numpy as np

x = np.array([1, 0, 0])
y = np.array([0, 1, 0])
z = np.array([0, 0, 1])

print("x: ", x)
print("y: ", y)
print("z: ", z)
print("<x, y>: ", np.dot(x, y))
print("<y, z>: ", np.dot(y, z))
print("<z, x>: ", np.dot(z, x))
print("||x||: ", np.linalg.norm(x))
print("||y||: ", np.linalg.norm(y))
print("||z||: ", np.linalg.norm(z))

### The Gram-Schmidt Process

If we have a set of vectors that are linearly independent, then that set has an orthonormal basis. This basis can be found for any set using the Gram-Schmidt process:

In [ ]:
import numpy as np

def proj(u, v):
    return np.dot(u, v) / np.dot(u, u) * u

def gram_schmidt(A):
    U = np.zeros(A.shape)
    E = np.zeros(A.shape)
    U[0] = A[0]
    E[0] = U[0] / np.linalg.norm(U[0])
    for i in range(1, A.shape[0]):
        U[i] = A[i]
        for j in range(i):
            U[i] -= proj(E[j], A[i])
        E[i] = U[i] / np.linalg.norm(U[i])

    return U, E

A = np.array([[1, 0, 0], [1, 1, 0], [1, 1, 1]])
U, E = gram_schmidt(A)
print("A:\n", A)
print("U:\n", U)
print("E:\n", E)

The Gram-Schmidt process takes in a set of linearly independent vectors A and returns a set of orthogonal vectors U and their corresponding orthonormal set E. This above example is quite simple, as the orthogonal vectors in U are already normalized, but for larger/more complex sets of vectors, U and E are likely to differ.

### Eigenvalues and Eigenvectors

In certain cases, a square matrix of values A may have corresponding eigenvalues λ and eigenvectors *x* such that A*x* = λ*x*. A simple method for determining the eigenvalues of any A algebraically is to find all λ that satisfy the equation det(A-λI) = 0 (where I is the identity matrix of same size as A). However, since eigenvalues and eigenvectors are very common in applied linear algebra, Python functions do exist to determine the eigenvalues/vectors for you:

In [ ]:
import numpy as np

#matrix with real eigenvalues
A = np.array([[1, 2], [2, 1]])

print("A:\n", A)
print("Eigenvalues:\n", np.linalg.eigvals(A))
print("Eigenvectors:\n", np.linalg.eig(A).eigenvectors)

#matrix with complex eigenvalues
B = np.array([[0, -1], [1, 0]])

print("B:\n", B)
print("Eigenvalues:\n", np.linalg.eigvals(B))
print("Eigenvectors:\n", np.linalg.eig(B).eigenvectors)

## Section 1.3: Linear Regression

Linear regression attempts to fit data using a function, similar to linear interpolation but instead with the goal to simply model existing data as best as possible, with interpolation as more of an afterthought. Given a set of independent vectors *x1, x2, ..., xn* and a response to the vectors *y*, the goal is to find coeffecients *β1, β2, ..., βn* that minimize the difference between the true values and a prediction model. This is best framed as a least-squares problem searching for *min(β) ||y-Aβ||^2* and can be solved with a QR decomposition, where A is decomposed into two matrices Q (where Q * Q^T = I) and R (an upper triangular matrix) such that A = QR:

In [ ]:
import numpy as np

def proj(u, v):
    return np.dot(u, v) / np.dot(u, u) * u

def gram_schmidt(A):
    U = np.zeros(A.shape)
    E = np.zeros(A.shape)
    U[0] = A[0]
    E[0] = U[0] / np.linalg.norm(U[0])
    for i in range(1, A.shape[0]):
        U[i] = A[i]
        for j in range(i):
            U[i] -= proj(E[j], A[i])
        E[i] = U[i] / np.linalg.norm(U[i])

    return U, E

def qr(A):
    U, E = gram_schmidt(A)
    Q = E.T
    R = np.zeros((A.shape[1], A.shape[1]))
    for i in range(A.shape[1]):
        for j in range(i, A.shape[1]):
            R[i, j] = np.dot(E[i], A[j])

    return Q, R

def linreg(X, Y):
    Q, R = qr(X)
    QTY = np.dot(Q.T, Y)
    β = np.linalg.solve(R, QTY)

    return β

X = np.array([[4, 1, 3], [2, 1, 2], [2, 3, 1]])
Y = np.array([5, 3, 7])
β = linreg(X, Y)
print("β: ", β)